In [1]:
## Ran for eveything now with excluding the gauges that we don't want

# Create a set of dimensionless profiles
Read in all of the events, for all durations, for all gauges, for all ensemble members.  
Convert them to dimensionless profiles, with 12 values between 0 and 1.  
Each value is a dimensionless, cumulative rainfall value (cumulative rainfall at this timestep, normalised by the total event rainfall):
- 0 means no rainfall has occurred, and 
- 1 means the total event rainfall has been reached.  

If there are less than 12 values, then these are filled in with interpolation

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os
import pandas as pd
import re
from datetime import datetime

from Create_Profiles_Functions import *

quintile_mapping = {1: 'F2', 2: 'F1', 3: 'C', 4: 'B1', 5: 'B2'}

In [2]:
gauge_nums = range(0,1294)

# NIMROD - 30 mins
### Make profiles

In [5]:
# # Initialize an empty DataFrame with the desired columns
# Initialize an empty DataFrame with the desired columns
columns = [
    'gauge_num',  'season', 'precip', 'Volume','Year', 'times', 'duration',
    'normalized_rainfall', 'cumulative_normalized_rainfall', 'cumulative_rainfall',
    'interpolated15_cumulative_rainfall', 'interpolated12_cumulative_rainfall','interpolated15_incremental_rainfall','interpolated12_incremental_rainfall',
    'interpolated15_cumulative_normalized_rainfall','interpolated12_cumulative_normalized_rainfall','interpolated15_incremental_normalized_rainfall','interpolated12_incremental_normalized_rainfall',
    'max_quintile_raw_rain','max_quintile_profile_12', 'max_quintile_profile_15', 'max_quintile_normalised_rain', 'max_quintile_profile_12_norm', "max_quintile_profile_15_norm"]
df = pd.DataFrame(columns=columns)

for nimrod_option in ["NIMROD_2.2km_filtered_100"]:
    for gauge_num in range(0, 1293):
        if gauge_num not in [444, 827, 888]:
            print(f"gauge {gauge_num}")
            files = [f for f in os.listdir(f"/nfs/a161/gy17m2a/PhD/ProcessedData/IndependentEvents/NIMROD_30mins/{nimrod_option}/{gauge_num}/WholeYear") if f.endswith('.csv')]
            files = np.sort(files)

            for file in files:
                fp = f"/nfs/a161/gy17m2a/PhD/ProcessedData/IndependentEvents/NIMROD_30mins/{nimrod_option}/{gauge_num}/WholeYear/{file}"
                if '2080' in fp:
                    continue

                this_event = read_event(gauge_num, fp)
                trimmed_event = remove_leading_and_trailing_zeroes(this_event)
                real_trimmed_event, problem_events = remove_events_with_problems(trimmed_event, verbose=False)

                if real_trimmed_event is not None:
                    
                    precip = real_trimmed_event['precipitation (mm/hr)']
                    normalized_rainfall = create_normalised_event(real_trimmed_event['precipitation (mm/hr)'])
                    cumulative_normalized_rainfall = create_cumulative_event(normalized_rainfall)
                    cumulative_rainfall = create_cumulative_event(precip)
                    
                    interpolated15_cumulative_rainfall = interpolate_rainfall(cumulative_rainfall,15)
                    interpolated12_cumulative_rainfall = interpolate_rainfall(cumulative_rainfall,12)
                    interpolated15_incremental_rainfall = create_incremental_event(interpolated15_cumulative_rainfall)
                    interpolated12_incremental_rainfall = create_incremental_event(interpolated12_cumulative_rainfall)
                    
                    interpolated15_cumulative_normalized_rainfall = interpolate_rainfall(cumulative_normalized_rainfall,15)
                    interpolated12_cumulative_normalized_rainfall = interpolate_rainfall(cumulative_normalized_rainfall,12)
                    interpolated15_incremental_normalized_rainfall = create_incremental_event(interpolated15_cumulative_normalized_rainfall)
                    interpolated12_incremental_normalized_rainfall = create_incremental_event(interpolated12_cumulative_normalized_rainfall)
                    
                    max_quintile_raw_rain = find_part_with_most_rain(precip, 5)
                    max_quintile_profile_12 = find_part_with_most_rain(interpolated12_incremental_rainfall, 5)
                    max_quintile_profile_15 = find_part_with_most_rain(interpolated15_incremental_rainfall, 5)
                    
                    max_quintile_normalised_rain = find_part_with_most_rain(normalized_rainfall, 5)
                    max_quintile_profile_12_norm = find_part_with_most_rain(interpolated12_incremental_normalized_rainfall, 5)
                    max_quintile_profile_15_norm = find_part_with_most_rain(interpolated15_incremental_normalized_rainfall, 5)
                    
                    duration = len(real_trimmed_event) / 2
                    times = trimmed_event['times'].values
                    season = get_season(trimmed_event['times'][0])
                    year=extract_year(trimmed_event)
                    
                else:
                    precip=trimmed_event['precipitation (mm/hr)']
                    normalized_rainfall = None
                    cumulative_normalized_rainfall = None
                    cumulative_rainfall = None
                    
                    interpolated15_cumulative_rainfall = None
                    interpolated12_cumulative_rainfall = None
                    interpolated15_incremental_rainfall = None
                    interpolated12_incremental_rainfall = None
                    
                    interpolated15_cumulative_normalized_rainfall = None
                    interpolated12_cumulative_normalized_rainfall = None
                    interpolated15_incremental_normalized_rainfall = None
                    interpolated12_incremental_normalized_rainfall = None
                    
                    max_quintile_raw_rain = None
                    max_quintile_profile_12 = None
                    max_quintile_profile_15 = None
                    
                    max_quintile_normalised_rain = None
                    max_quintile_profile_12_norm = None
                    max_quintile_profile_15_norm = None
                    
                    duration = None
                    season = None
                    year=None
                    times=None
                    
                # Append the row to the DataFrame
                df = df.append({
                    'gauge_num': gauge_num,
                    'season': season,
                    'precip': precip.values,
                    
                    'Volume': sum(precip),
                    'Year':year,
                    'times':times, 
                    'duration': duration,
                    
                    'normalized_rainfall':normalized_rainfall,
                    'cumulative_normalized_rainfall':cumulative_normalized_rainfall,
                    'cumulative_rainfall':cumulative_rainfall,
                    
                    'interpolated15_cumulative_rainfall':interpolated15_cumulative_rainfall,
                    'interpolated12_cumulative_rainfall':interpolated12_cumulative_rainfall,                    
                    'interpolated15_incremental_rainfall':interpolated15_incremental_rainfall,
                    'interpolated12_incremental_rainfall': interpolated12_incremental_rainfall,
                    
                    'interpolated15_cumulative_normalized_rainfall':interpolated15_cumulative_normalized_rainfall,
                    "interpolated12_cumulative_normalized_rainfall":interpolated12_cumulative_normalized_rainfall,
                    'interpolated15_incremental_normalized_rainfall':interpolated15_incremental_normalized_rainfall,
                    'interpolated12_incremental_normalized_rainfall':interpolated12_incremental_normalized_rainfall,
                    
                    'max_quintile_raw_rain': max_quintile_raw_rain,
                    'max_quintile_profile_12': max_quintile_profile_12,
                    'max_quintile_profile_15': max_quintile_profile_15,
                    'max_quintile_normalised_rain': max_quintile_normalised_rain,
                    'max_quintile_profile_12_norm': max_quintile_profile_12_norm,
                    'max_quintile_profile_15_norm': max_quintile_profile_15_norm                    
                }, ignore_index=True)
                

df['Loading_profile_raw_rain'] = df['max_quintile_raw_rain'].map(quintile_mapping)
df['Loading_profile12'] = df['max_quintile_profile_12'].map(quintile_mapping)
df['Loading_profile15'] = df['max_quintile_profile_15'].map(quintile_mapping)
df['Loading_profile_normalised_rain'] = df['max_quintile_normalised_rain'].map(quintile_mapping)
df['Loading_profile_norm_12'] = df['max_quintile_profile_12_norm'].map(quintile_mapping)
df['Loading_profile_norm_15'] = df['max_quintile_profile_15_norm'].map(quintile_mapping)

df = add_duration_cats_based_on_data(df)
df = add_duration_cats_predetermined(df)

# with open(f"/nfs/a319/gy17m2a/PhD/ProcessedData/Profiles/NIMROD_30mins/WholeYear/{nimrod_option}_profiles_df.pkl", 'wb') as file:
#     pickle.dump(df, file)

gauge 0
gauge 1
gauge 2
gauge 3
gauge 4
gauge 5
gauge 6
gauge 7
gauge 8
gauge 9
gauge 10
gauge 11
gauge 12
gauge 13
gauge 14
gauge 15
gauge 16
gauge 17
gauge 18
gauge 19
gauge 20
gauge 21
gauge 22
gauge 23
gauge 24
gauge 25
gauge 26
gauge 27


PermissionError: [Errno 13] Permission denied: '/nfs/a161/gy17m2a/PhD/ProcessedData/IndependentEvents/NIMROD_30mins/NIMROD_2.2km_filtered_100/27/WholeYear/0.5hrs_2016_v2_part0.csv'

# NIMROD 5 mins

In [22]:
# Initialize an empty DataFrame with the desired columns
# columns = [
#     'gauge_num',  'season', 'precip', 'Volume','Year', 'times', 'duration',
#     'normalized_rainfall', 'cumulative_normalized_rainfall', 'cumulative_rainfall',
#     'interpolated15_cumulative_rainfall', 'interpolated12_cumulative_rainfall','interpolated15_incremental_rainfall','interpolated12_incremental_rainfall',
#     'interpolated15_cumulative_normalized_rainfall','interpolated12_cumulative_normalized_rainfall','interpolated15_incremental_normalized_rainfall','interpolated12_incremental_normalized_rainfall',
#     'max_quintile_raw_rain','max_quintile_profile_12', 'max_quintile_profile_15', 'max_quintile_normalised_rain', 'max_quintile_profile_12_norm', "max_quintile_profile_15_norm"]
# df = pd.DataFrame(columns=columns)

for nimrod_option in ["NIMROD_1km_filtered_100"]:
    for gauge_num in range(350,1201):
        if gauge_num not in [444, 827, 888]:
            print(f"gauge {gauge_num}")
            files = [f for f in os.listdir(f"/nfs/a161/gy17m2a/PhD/ProcessedData/IndependentEvents/NIMROD_5mins/{nimrod_option}/{gauge_num}/WholeYear") if f.endswith('.csv')]
            files = np.sort(files)

            for file in files:
                fp = f"/nfs/a161/gy17m2a/PhD/ProcessedData/IndependentEvents/NIMROD_5mins/{nimrod_option}/{gauge_num}/WholeYear/{file}"
                if '2080' in fp:
                    continue

                this_event = read_event(gauge_num, fp)
                trimmed_event = remove_leading_and_trailing_zeroes(this_event)
                real_trimmed_event, problem_events = remove_events_with_problems(trimmed_event, verbose=False)

                if real_trimmed_event is not None:
                    
                    precip = real_trimmed_event['precipitation (mm/hr)']
                    normalized_rainfall = create_normalised_event(real_trimmed_event['precipitation (mm/hr)'])
                    cumulative_normalized_rainfall = create_cumulative_event(normalized_rainfall)
                    cumulative_rainfall = create_cumulative_event(precip)
                    
                    interpolated15_cumulative_rainfall = interpolate_rainfall(cumulative_rainfall,15)
                    interpolated12_cumulative_rainfall = interpolate_rainfall(cumulative_rainfall,12)
                    interpolated15_incremental_rainfall = create_incremental_event(interpolated15_cumulative_rainfall)
                    interpolated12_incremental_rainfall = create_incremental_event(interpolated12_cumulative_rainfall)
                    
                    interpolated15_cumulative_normalized_rainfall = interpolate_rainfall(cumulative_normalized_rainfall,15)
                    interpolated12_cumulative_normalized_rainfall = interpolate_rainfall(cumulative_normalized_rainfall,12)
                    interpolated15_incremental_normalized_rainfall = create_incremental_event(interpolated15_cumulative_normalized_rainfall)
                    interpolated12_incremental_normalized_rainfall = create_incremental_event(interpolated12_cumulative_normalized_rainfall)
                    
                    max_quintile_raw_rain = find_part_with_most_rain(precip, 5)
                    max_quintile_profile_12 = find_part_with_most_rain(interpolated12_incremental_rainfall, 5)
                    max_quintile_profile_15 = find_part_with_most_rain(interpolated15_incremental_rainfall, 5)
                    
                    max_quintile_normalised_rain = find_part_with_most_rain(normalized_rainfall, 5)
                    max_quintile_profile_12_norm = find_part_with_most_rain(interpolated12_incremental_normalized_rainfall, 5)
                    max_quintile_profile_15_norm = find_part_with_most_rain(interpolated15_incremental_normalized_rainfall, 5)
                    
                    duration = len(real_trimmed_event) / 2
                    times = trimmed_event['times'].values
                    season = get_season(trimmed_event['times'][0])
                    year=extract_year(trimmed_event)
                    
                else:
                    precip=trimmed_event['precipitation (mm/hr)']
                    normalized_rainfall = None
                    cumulative_normalized_rainfall = None
                    cumulative_rainfall = None
                    
                    interpolated15_cumulative_rainfall = None
                    interpolated12_cumulative_rainfall = None
                    interpolated15_incremental_rainfall = None
                    interpolated12_incremental_rainfall = None
                    
                    interpolated15_cumulative_normalized_rainfall = None
                    interpolated12_cumulative_normalized_rainfall = None
                    interpolated15_incremental_normalized_rainfall = None
                    interpolated12_incremental_normalized_rainfall = None
                    
                    max_quintile_raw_rain = None
                    max_quintile_profile_12 = None
                    max_quintile_profile_15 = None
                    
                    max_quintile_normalised_rain = None
                    max_quintile_profile_12_norm = None
                    max_quintile_profile_15_norm = None
                    
                    duration = None
                    season = None
                    year=None
                    times=None
                    
                # Append the row to the DataFrame
                df = df.append({
                    'gauge_num': gauge_num,
                    'season': season,
                    'precip': precip.values,
                    
                    'Volume': sum(precip),
                    'Year':year,
                    'times':times, 
                    'duration': duration,
                    
                    'normalized_rainfall':normalized_rainfall,
                    'cumulative_normalized_rainfall':cumulative_normalized_rainfall,
                    'cumulative_rainfall':cumulative_rainfall,
                    
                    'interpolated15_cumulative_rainfall':interpolated15_cumulative_rainfall,
                    'interpolated12_cumulative_rainfall':interpolated12_cumulative_rainfall,                    
                    'interpolated15_incremental_rainfall':interpolated15_incremental_rainfall,
                    'interpolated12_incremental_rainfall': interpolated12_incremental_rainfall,
                    
                    'interpolated15_cumulative_normalized_rainfall':interpolated15_cumulative_normalized_rainfall,
                    "interpolated12_cumulative_normalized_rainfall":interpolated12_cumulative_normalized_rainfall,
                    'interpolated15_incremental_normalized_rainfall':interpolated15_incremental_normalized_rainfall,
                    'interpolated12_incremental_normalized_rainfall':interpolated12_incremental_normalized_rainfall,
                    
                    'max_quintile_raw_rain': max_quintile_raw_rain,
                    'max_quintile_profile_12': max_quintile_profile_12,
                    'max_quintile_profile_15': max_quintile_profile_15,
                    'max_quintile_normalised_rain': max_quintile_normalised_rain,
                    'max_quintile_profile_12_norm': max_quintile_profile_12_norm,
                    'max_quintile_profile_15_norm': max_quintile_profile_15_norm                    
                }, ignore_index=True)
                

df['Loading_profile_raw_rain'] = df['max_quintile_raw_rain'].map(quintile_mapping)
df['Loading_profile12'] = df['max_quintile_profile_12'].map(quintile_mapping)
df['Loading_profile15'] = df['max_quintile_profile_15'].map(quintile_mapping)
df['Loading_profile_normalised_rain'] = df['max_quintile_normalised_rain'].map(quintile_mapping)
df['Loading_profile_norm_12'] = df['max_quintile_profile_12_norm'].map(quintile_mapping)
df['Loading_profile_norm_15'] = df['max_quintile_profile_15_norm'].map(quintile_mapping)

df = add_duration_cats_based_on_data(df)
df = add_duration_cats_predetermined(df)

with open(f"/nfs/a319/gy17m2a/PhD/ProcessedData/Profiles/NIMROD_5mins/WholeYear/{nimrod_option}_profiles_df.pkl", 'wb') as file:
    pickle.dump(df, file)

gauge 350
gauge 351
gauge 352
gauge 353
gauge 354
gauge 355
gauge 356
gauge 357
gauge 358
gauge 359
gauge 360
gauge 361
gauge 362
gauge 363
gauge 364
gauge 365
gauge 366
gauge 367
gauge 368
gauge 369
gauge 370
gauge 371
gauge 372
gauge 373
gauge 374
gauge 375
gauge 376
gauge 377
gauge 378
gauge 379
gauge 380
gauge 381
gauge 382
gauge 383
gauge 384
gauge 385
gauge 386
gauge 387
gauge 388
gauge 389
gauge 390
gauge 391
gauge 392
gauge 393
gauge 394
gauge 395
gauge 396
gauge 397
gauge 398
gauge 399
gauge 400
gauge 401
gauge 402
gauge 403
gauge 404
gauge 405
gauge 406
gauge 407
gauge 408
gauge 409
gauge 410
gauge 411
gauge 412
gauge 413
gauge 414
gauge 415
gauge 416
gauge 417
gauge 418
gauge 419
gauge 420
gauge 421
gauge 422
gauge 423
gauge 424
gauge 425
gauge 426
gauge 427
gauge 428
gauge 429
gauge 430
gauge 431
gauge 432
gauge 433
gauge 434
gauge 435
gauge 436
gauge 437
gauge 438
gauge 439
gauge 440
gauge 441
gauge 442
gauge 443
gauge 445
gauge 446
gauge 447
gauge 448
gauge 449
gauge 450


KeyboardInterrupt: 

In [23]:
with open(f"/nfs/a319/gy17m2a/PhD/ProcessedData/Profiles/NIMROD_5mins/WholeYear/{nimrod_option}_profiles_df.pkl", 'wb') as file:
    pickle.dump(df, file)

In [20]:
with open(f"/nfs/a319/gy17m2a/PhD/ProcessedData/Profiles/NIMROD_5mins/WholeYear/{nimrod_option}_profiles_df.pkl", 'wb') as file:
    pickle.dump(df, file)

# UKCP18
### Make and pickle profiles

In [273]:
# for em in ["bc005"]:

#     # New list to store just the files that are 'good' e.g. 
#     dict_of_things_im_storing ={}
#     dict_of_things_im_storing['good_raw_events'] = []
#     dict_of_things_im_storing['good_raw_events_seasons'] = []
#     dict_of_things_im_storing['good_trimmed_events'] = []
#     dict_of_things_im_storing['good_trimmed_events_durations'] = []
#     dict_of_things_im_storing['good_normalised_events'] = []
#     dict_of_things_im_storing['good_normalised_interpolated_events'] = []
#     dict_of_things_im_storing['good_normalised_interpolated_events_max_quintiles'] = []

#     for gauge_num in range(0,1294):
#         if gauge_num not in [444, 827, 888]:
#             print(f"gauge {gauge_num}")
#             files = [f for f in os.listdir(f"/nfs/a161/gy17m2a/PhD/ProcessedData/IndependentEvents/UKCP18_30mins/{em}/{gauge_num}/WholeYear/") if f.endswith('.csv')]
#             files = np.sort(files)

#             # Go through each event
#             for file in files:
#                 # Construct the path to the file
#                 fp = f"/nfs/a161/gy17m2a/PhD/ProcessedData/IndependentEvents/UKCP18_30mins/{em}/{gauge_num}/WholeYear/{file}"
#                 if '2080' in fp:
#                     pass
#                     # print(f"{fp} 2080 so not using")
#                 else:
#                     # print(f"{fp}not 2080 so using")
#                     # Read in event
#                     this_event = read_event(gauge_num, fp)
#                     # Trim leading and trailing zeroes
#                     trimmed_event = remove_leading_and_trailing_zeroes(this_event)
                    
#                     # Check trimmed event for 'problems', if there are none, return a dataframe, if there are, return None
#                     real_trimmed_event, problem_events = remove_events_with_problems(trimmed_event, verbose=False)

#                     # If we still have a dataframe, then add it to list of good, unprocessed events
#                     if real_trimmed_event is not None:

#                         # Create normalised version of event
#                         normalized_time, normalized_rainfall = create_normalised_event(real_trimmed_event)
#                         print(len(normalized_rainfall))
#                         # Create interpolated, normalised version of event (with 12 data points)
#                         normalized_interpolated_rainfall_12 = interpolate_and_bin(normalized_time, normalized_rainfall, 12)
#                         normalized_interpolated_rainfall_15 = interpolate_and_bin(normalized_time, normalized_rainfall, 15)
                        
#                         # Find the fifth OF THE INTERPOLATED, NORMALISED EVENT with the maximum amount of rainfall
#                         max_quintile_profile_12 = find_part_with_most_rain(normalized_interpolated_rainfall_12, 5)
#                         max_quintile_profile_15 = find_part_with_most_rain(normalized_interpolated_rainfall_15, 5)
#                         max_quintile_raw_rain = find_part_with_most_rain(normalized_rainfall, 5)
                        
#                         duration = len(real_trimmed_event)/2
#                         season = get_season(trimmed_event['times'][0])
                                            
#                     else:
#                         print(f"{file}event not real")
#                         normalized_rainfall = None
#                         normalized_interpolated_rainfall_12 = None
#                         normalized_interpolated_rainfall_15 = None
#                         max_quintile_profile_12 = None
#                         max_quintile_profile_15 = None
#                         max_quintile_raw_rain = None
#                         duration = None
#                         season = None

#                     # Add to list of events with no problems
#                     dict_of_things_im_storing['good_raw_events'].append(this_event)
#                     dict_of_things_im_storing['good_raw_events_seasons'].append(season)
#                     dict_of_things_im_storing['good_trimmed_events'].append(real_trimmed_event)
#                     dict_of_things_im_storing['good_trimmed_events_durations'].append(duration)
#                     dict_of_things_im_storing['good_normalised_events'].append(normalized_rainfall)
#                     dict_of_things_im_storing['good_normalised_interpolated_events'].append(normalized_interpolated_rainfall)
#                     dict_of_things_im_storing['good_normalised_interpolated_events_max_quintiles'].append(max_quintile)

#         print(len(dict_of_things_im_storing['good_normalised_interpolated_events_max_quintiles']))
        
#     for file_name,stored_list in dict_of_things_im_storing.items():
#         print(file_name)
#         cache_filepath = f"/nfs/a319/gy17m2a/PhD/ProcessedData/Profiles/UKCP18_30mins/{em}/new_{file_name}.pkl"
#         with open(cache_filepath, 'wb') as f:
#             pickle.dump(stored_list, f)                        

In [15]:
# # # Initialize an empty DataFrame with the desired columns
# columns = [
#     'gauge_num',  'season', 'precip', 'Volume','Year', 'duration',
#     'normalized_rainfall', 'normalized_interpolated_rainfall_12', 'normalized_interpolated_rainfall_15',
#     'max_quintile_profile_12', 'max_quintile_profile_15', 'max_quintile_normalised_rain', 'max_quintile_raw_rain']
# df = pd.DataFrame(columns=columns)

# for em in ["bb198"]:
#     for gauge_num in range(0, 1293):
#         if gauge_num not in [444, 827, 888]:
#             print(f"gauge {gauge_num}")
#             files = [f for f in os.listdir(f"/nfs/a161/gy17m2a/PhD/ProcessedData/IndependentEvents/UKCP18_30mins/{em}/{gauge_num}/WholeYear/") if f.endswith('.csv')]
#             files = np.sort(files)

#             for file in files:
#                 fp = f"/nfs/a161/gy17m2a/PhD/ProcessedData/IndependentEvents/UKCP18_30mins/{em}/{gauge_num}/WholeYear/{file}"
#                 if '2080' in fp:
#                     continue

#                 this_event = read_event(gauge_num, fp)
#                 trimmed_event = remove_leading_and_trailing_zeroes(this_event)
#                 real_trimmed_event, problem_events = remove_events_with_problems(trimmed_event, verbose=False)

#                 if real_trimmed_event is not None:
#                     precip = real_trimmed_event['precipitation (mm/hr)']
#                     normalized_time, normalized_rainfall = create_normalised_event(real_trimmed_event)
#                     normalized_interpolated_rainfall_12 = interpolate_and_bin(normalized_time, normalized_rainfall, 12)
#                     normalized_interpolated_rainfall_15 = interpolate_and_bin(normalized_time, normalized_rainfall, 15)
#                     max_quintile_profile_12 = find_part_with_most_rain_using_cumulative_rainfall(normalized_interpolated_rainfall_12, 5)
#                     max_quintile_profile_15 = find_part_with_most_rain_using_cumulative_rainfall(normalized_interpolated_rainfall_15, 5)
#                     max_quintile_normalised_rain = find_part_with_most_rain_using_cumulative_rainfall(normalized_rainfall, 5)
#                     max_quintile_raw_rain = find_part_with_most_rain(real_trimmed_event['precipitation (mm/hr)'], 5)
#                     duration = len(real_trimmed_event) / 2
#                     season = get_season(trimmed_event['times'][0])
#                 else:
#                     precip=trimmed_event['precipitation (mm/hr)']
#                     normalized_rainfall = None
#                     normalized_interpolated_rainfall_12 = None
#                     normalized_interpolated_rainfall_15 = None
#                     max_quintile_profile_12 = None
#                     max_quintile_profile_15 = None
#                     max_quintile_normalised_rain = None
#                     max_quintile_raw_rain = None
#                     duration = None
#                     season = None
#                 # Append the row to the DataFrame
#                 df = df.append({
#                     'gauge_num': gauge_num,
#                     'season': season,
#                     'precip': precip.values,
#                     'Volume': sum(precip),
#                     'Year':extract_year(trimmed_event),
#                     'duration': duration,
#                     'normalized_rainfall': normalized_rainfall,
#                     'normalized_interpolated_rainfall_12': normalized_interpolated_rainfall_12,
#                     'normalized_interpolated_rainfall_15': normalized_interpolated_rainfall_15,
#                     'max_quintile_profile_12': max_quintile_profile_12,
#                     'max_quintile_profile_15': max_quintile_profile_15,
#                     'max_quintile_normalised_rain': max_quintile_normalised_rain,
#                     'max_quintile_raw_rain': max_quintile_raw_rain
#                 }, ignore_index=True)
# df['Loading_profile12'] = df['max_quintile_profile_12'].map(quintile_mapping)
# df['Loading_profile15'] = df['max_quintile_profile_15'].map(quintile_mapping)
# df['Loading_profile_normalised_rain'] = df['max_quintile_normalised_rain'].map(quintile_mapping)
# df['Loading_profile_raw_rain'] = df['max_quintile_raw_rain'].map(quintile_mapping)

with open(f"/nfs/a319/gy17m2a/PhD/ProcessedData/Profiles/UKCP18_30mins/{em}/profiles_df.pkl", 'wb') as file:
    pickle.dump(df, file)
    
    
    #         with open(cache_filepath, 'wb') as f:
#             pickle.dump(stored_list, f)   